
### Path setup (root results)
Ensures we resolve the repo root and use the root `results/` directory (with legacy fallback under `src/.../results`).


In [1]:

from pathlib import Path
import sys
import pandas as pd

# Locate repo root (assumes running somewhere inside the repo)
cwd = Path.cwd().resolve()
REPO_ROOT = cwd
for parent in [cwd] + list(cwd.parents):
    if (parent / "src" / "motion_planning").exists():
        REPO_ROOT = parent
        break

# Ensure src on path
src_path = REPO_ROOT / "src"
if str(src_path) not in sys.path:
    sys.path.insert(0, str(src_path))

print("Repo root:", REPO_ROOT)
print("sys.path[0]:", sys.path[0])


Repo root: /home/abdulrahman/competitive-online-algorithms-motion-planning
sys.path[0]: /home/abdulrahman/competitive-online-algorithms-motion-planning/src


In [2]:

# Candidate locations to search (new root results + legacy src path)
SEARCH_BASES = [
    REPO_ROOT / "results",
    REPO_ROOT / "src/motion_planning/offline_problems/results",
]

def find_files(pattern: str):
    hits = []
    for base in SEARCH_BASES:
        if base.exists():
            hits.extend(base.rglob(pattern))
    return sorted(set(hits))

cand_files = find_files("*_candidates.csv")
edge_files = find_files("*_edge_values.csv")
timeline_files = find_files("*_decision_timeline.csv")
opt_solution_files = find_files("*_solution_edges.csv") + find_files("*_opt_solution.csv")

print("Found candidates:", cand_files[-2:])
print("Found edges:", edge_files[-2:])
print("Found timelines:", timeline_files[-2:])
print("Found offline solutions:", opt_solution_files[-2:])

sample_cand = cand_files[-1] if cand_files else None
sample_edge = edge_files[-1] if edge_files else None
sample_timeline = timeline_files[-1] if timeline_files else None
sample_opt = opt_solution_files[-1] if opt_solution_files else None
print("Using sample cand:", sample_cand)
print("Using sample edge:", sample_edge)
print("Using sample timeline:", sample_timeline)
print("Using sample opt solution:", sample_opt)

from motion_planning.online_algorithms.load_problem_to_solve import (
    load_candidates_problem,
    load_graph_problem,
)
from motion_planning.online_algorithms.orb_czl import run_czl_orb
from motion_planning.online_algorithms.orb_bat import run_bat_orb
from motion_planning.online_algorithms.orb_itm import run_itm_online
from motion_planning.online_algorithms.thresholds import czl_thresholds, bat_threshold

# CAND_PATH = REPO_ROOT / "src/motion_planning/offline_problems/results/data/offline problems/problem details"
# EDGE_PATH = REPO_ROOT / "src/motion_planning/offline_problems/results/data/offline problems/graphs with risks"
# TIMELINE_PATH = REPO_ROOT / "src/motion_planning/offline_problems/results/data/offline problems/problem details"
# print("Repo root:", REPO_ROOT)
# print("sys.path[0]:", sys.path[0])
# print(CAND_PATH)

Found candidates: [PosixPath('/home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/offline problems/problem details/20251125_164202_Town03_Roundabout_10_mid_5_candidates.csv'), PosixPath('/home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/offline problems/problem details/20251125_164202_Town03_Roundabout_15_mid_5_candidates.csv')]
Found edges: [PosixPath('/home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/offline problems/graphs with risks/20251125_164202_Town03_Roundabout_10_mid_5_edge_values.csv'), PosixPath('/home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/offline problems/graphs with risks/20251125_164202_Town03_Roundabout_15_mid_5_edge_values.csv')]
Found timelines: [PosixPath('/home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/offline problems/problem details/20251125_164202_Town03_Roundabout_10_mid_5_decision_timeline.csv'), PosixPath('/home/abdulrahm

In [3]:

# List available candidates/edge files (fallback to rglob if empty)
# cand_files = sorted(CAND_PATH.glob("*_candidates.csv"))
# edge_files = sorted(EDGE_PATH.glob("*_edge_values.csv"))
# timeline_files = sorted(TIMELINE_PATH.glob("*_decision_timeline.csv"))

if not cand_files:
    cand_files = sorted((REPO_ROOT / "src").rglob("*_candidates.csv"))
if not edge_files:
    edge_files = sorted((REPO_ROOT / "src").rglob("*_edge_values.csv"))
if not timeline_files:
    timeline_files = sorted((REPO_ROOT / "src").rglob("*_decision_timeline.csv"))

print("Found candidates:", cand_files[-2:])
print("Found edge files:", edge_files[-2:])
print("Found timelines:", timeline_files[-2:])

sample_cand = cand_files[-1] if cand_files else None
sample_edge = edge_files[-1] if edge_files else None
sample_timeline = timeline_files[-1] if timeline_files else None
print("Using sample cand:", sample_cand)
print("Using sample edge:", sample_edge)
print("Using sample timeline:", sample_timeline)


Found candidates: [PosixPath('/home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/offline problems/problem details/20251125_164202_Town03_Roundabout_10_mid_5_candidates.csv'), PosixPath('/home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/offline problems/problem details/20251125_164202_Town03_Roundabout_15_mid_5_candidates.csv')]
Found edge files: [PosixPath('/home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/offline problems/graphs with risks/20251125_164202_Town03_Roundabout_10_mid_5_edge_values.csv'), PosixPath('/home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/offline problems/graphs with risks/20251125_164202_Town03_Roundabout_15_mid_5_edge_values.csv')]
Found timelines: [PosixPath('/home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/offline problems/problem details/20251125_164202_Town03_Roundabout_10_mid_5_decision_timeline.csv'), PosixPath('/home/abdu

In [4]:

if sample_edge:
    df_edges = pd.read_csv(sample_edge)
    print("Edges shape:", df_edges.shape)
    display(df_edges.head())
else:
    print("No edge_values.csv found.")


Edges shape: (3240, 7)


,graph_id,start_node,end_node,speed,risk,cost,utility
0,Town03_Roundabout,v_5_6,v_6_5,10,2.445142,13.855963,86.144037
1,Town03_Roundabout,v_5_6,v_6_5,15,4.202929,12.570642,87.429358
2,Town03_Roundabout,v_5_6,v_6_5,20,0.851286,11.927981,88.072019
3,Town03_Roundabout,v_5_6,v_6_5,25,0.730436,11.542385,88.457615
4,Town03_Roundabout,v_5_6,v_6_5,30,1.124502,11.285321,88.714679


In [5]:
if sample_cand:
    rho_lo, rho_hi, rho_mid = czl_thresholds(sample_cand)
    rmin = bat_threshold(sample_cand)
    print("CZL thresholds ->", rho_lo, rho_hi, rho_mid)
    print("BAT risk_min ->", rmin)
else:
    print("Skip thresholds: no candidates file.")


CZL thresholds -> 31.740156047693176 57718.80071470702 28875.270435377355
BAT risk_min -> 0.001


In [6]:
from motion_planning.utils.paths import ONLINE_RESULTS_CANDIDATES_DIR

if sample_cand:
    czl_out = run_czl_orb(sample_cand, output_root=ONLINE_RESULTS_CANDIDATES_DIR)
    bat_out = run_bat_orb(sample_cand, output_root=ONLINE_RESULTS_CANDIDATES_DIR)
    print("CZL output:", czl_out)
    print("BAT output:", bat_out)
else:
    print("Skip CZL/BAT: no candidates file.")


CZL output: /home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/online solutions/candidates/20251125_164202_Town03_Roundabout_15_mid_5_online_CZL-ORB.csv
BAT output: /home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/online solutions/candidates/20251125_164202_Town03_Roundabout_15_mid_5_online_BAT-ORB.csv


In [7]:

if sample_edge and sample_timeline:
    graph_problem = load_graph_problem(sample_edge, sample_timeline)
    print("Graph graph_id:", graph_problem.graph_id)
    print("Edges df shape:", graph_problem.edges_df.shape)
    print("Timeline shape:", graph_problem.timeline_df.shape)
else:
    print("No edge/timeline file found. Update paths or generate offline problems.")


Graph graph_id: Town03_Roundabout
Edges df shape: (3240, 7)
Timeline shape: (6, 4)


In [8]:
from motion_planning.constrained_shortest_path.gurobi_license import GUROBI_OPTIONS
if sample_edge and sample_timeline:
    itm_res = run_itm_online(sample_edge, sample_timeline)
    print("ITM result:", itm_res)
else:
    print("Skip ITM: no edge/timeline file.")


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2649771
Academic license 2649771 - for non-commercial use only - registered to ab___@gmail.com
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2649771
Academic license 2649771 - for non-commercial use only - registered to ab___@gmail.com
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2649771
Academic license 2649771 - for non-commercial use only - registered to ab___@gmail.com
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2649771
Academic license 2649771 - for non-commercial use only - registered to ab___@gmail.com
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2649771
Academic license 2649771 - for non-commercial use only - registered to ab___@gmail.com
ITM result: [{'epoch': 0, 'start': 'v_0_0', 'goal': 'v_4_5', 'psi': 11.676550869519742, 'status': 2, 'selected


## Smoke Checks (added)
Load offline artifacts, compute thresholds, and run CZL/BAT/ITM on sample data.



### CZL-ORB run
Arguments: candidates_csv, capacity_override (Δ0), rho_min, rho_max, output_root.


In [ ]:

# Prepare args for CZL-ORB
delta0 = None
if sample_cand:
    # Parse budget from filename (token before risk label)
    parts = Path(sample_cand).stem.split("_")
    risk_labels = {"low","mid","medium","high","veryhigh","very_high"}
    for idx, tok in enumerate(parts):
        if tok.lower() in risk_labels and idx > 0:
            try:
                delta0 = float(parts[idx-1]); break
            except Exception:
                pass
    if delta0 is None:
        delta0 = float(5.0)
    rho_min, rho_max, _ = czl_thresholds(cand_files)
    czl_out = run_czl_orb(
        sample_cand,
        capacity_override=delta0,
        rho_min=rho_min,
        rho_max=rho_max,
    )
    print("CZL-ORB output:", czl_out)
else:
    print("No candidates file available for CZL-ORB.")



### BAT-ORB run
Arguments: candidates_csv, capacity_override (Δ0), delta_min, output_root.


In [ ]:

# Prepare args for BAT-ORB
delta0_bat = delta0 if 'delta0' in locals() else None
if sample_cand:
    if delta0_bat is None:
        parts = Path(sample_cand).stem.split("_")
        risk_labels = {"low","mid","medium","high","veryhigh","very_high"}
        for idx, tok in enumerate(parts):
            if tok.lower() in risk_labels and idx > 0:
                try:
                    delta0_bat = float(parts[idx-1]); break
                except Exception:
                    pass
    if delta0_bat is None:
        delta0_bat = float(5.0)
    delta_min = bat_threshold(cand_files)
    bat_out = run_bat_orb(
        sample_cand,
        capacity_override=delta0_bat,
        delta_min=delta_min,
    )
    print("BAT-ORB output:", bat_out)
else:
    print("No candidates file available for BAT-ORB.")



### ITM-ORB run
Arguments: edge_values_csv, decision_timeline_csv, capacity (Δ0), psi_t (None to auto-CZL), gurobi_params, output_root.


In [ ]:

# Prepare args for ITM-ORB
if sample_edge and sample_timeline:
    delta0_itm = None
    parts = Path(sample_edge).stem.split("_")
    risk_labels = {"low","mid","medium","high","veryhigh","very_high"}
    for idx, tok in enumerate(parts):
        if tok.lower() in risk_labels and idx > 0:
            try:
                delta0_itm = float(parts[idx-1]); break
            except Exception:
                pass
    if delta0_itm is None:
        delta0_itm = float(5.0)
    itm_res = run_itm_online(
        sample_edge,
        sample_timeline,
        capacity=delta0_itm,
        psi_t=None,
        gurobi_params=GUROBI_OPTIONS,
    )
    print("ITM-ORB result count:", len(itm_res))
    if itm_res:
        print("First epoch result:", itm_res[0])
else:
    print("No edge/timeline file available for ITM-ORB.")



### Refined artifact loading (using helpers)
Load offline artifacts via the helper modules and set up inputs for algorithms without fallbacks.


In [2]:
import sys
from pathlib import Path

sys.path.insert(0, str(Path.cwd().resolve().parents[1]))
import sys
import pandas as pd

from motion_planning.utils.paths import OFFLINE_RESULTS_DIR
from motion_planning.online_algorithms.load_problem_to_solve import load_candidates_problem, load_graph_problem
from motion_planning.online_algorithms.thresholds import czl_thresholds, bat_threshold
from motion_planning.online_algorithms.orb_itm import _parse_risk_budget_from_stem
from motion_planning.constrained_shortest_path.gurobi_license import GUROBI_OPTIONS
from motion_planning.online_algorithms.orb_czl import run_czl_orb
from motion_planning.online_algorithms.orb_bat import run_bat_orb
from motion_planning.online_algorithms.orb_itm import run_itm_online

cand_dir = OFFLINE_RESULTS_DIR / "problem details"
edge_dir = OFFLINE_RESULTS_DIR / "graphs with risks"
solution_dir = OFFLINE_RESULTS_DIR / "solution details"

cand_files = sorted(cand_dir.glob("*_candidates.csv"))
cand_opt_files = sorted(cand_dir.glob("*_offline_candidates_opt.csv"))
edge_files = sorted(edge_dir.glob("*_edge_values.csv"))
timeline_files = sorted(cand_dir.glob("*_decision_timeline.csv"))
graph_opt_files = sorted(solution_dir.glob("*_offline_graph_solution_edges.csv"))

print("Counts -> candidates:", len(cand_files), "cand opt:", len(cand_opt_files), "edges:", len(edge_files), "timelines:", len(timeline_files), "graph opts:", len(graph_opt_files))

# Load candidate problems
test_candidates = [load_candidates_problem(f) for f in cand_files]

# Map timelines by base stem
timeline_map = {}
for t in timeline_files:
    base = t.stem.replace("_decision_timeline", "")
    timeline_map[base] = t

# Load graph problems (only if timeline exists)
test_graphs = []
for ef in edge_files:
    base = ef.stem.replace("_edge_values", "")
    tl = timeline_map.get(base)
    if tl is None:
        continue
    gp = load_graph_problem(ef, tl)
    test_graphs.append(gp)

print("Loaded candidate problems:", len(test_candidates))
print("Loaded graph problems:", len(test_graphs))


Counts -> candidates: 2 cand opt: 2 edges: 2 timelines: 2 graph opts: 2
Loaded candidate problems: 2
Loaded graph problems: 2



### CZL-ORB (using helpers, no fallbacks)


In [6]:
from motion_planning.utils.paths import ONLINE_RESULTS_CANDIDATES_DIR
if test_candidates:
    rho_min, rho_max, _ = czl_thresholds(cand_files)
    for cand_prob in test_candidates:
    # cand_prob = test_candidates[0]
        delta0 = cand_prob.capacity
        czl_out = run_czl_orb(
            cand_prob.source_path,
            capacity_override=delta0,
            rho_min=rho_min,
            rho_max=rho_max,
            output_root=ONLINE_RESULTS_CANDIDATES_DIR
        )
        print("CZL-ORB output path:", czl_out)
    else:
        print("finished the CZL successfully!.")


CZL-ORB output path: /home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/online solutions/candidates/20251125_164202_Town03_Roundabout_10_mid_5_online_CZL-ORB.csv
CZL-ORB output path: /home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/online solutions/candidates/20251125_164202_Town03_Roundabout_15_mid_5_online_CZL-ORB.csv
finished the CZL successfully!.



### BAT-ORB (using helpers, no fallbacks)


In [8]:
from motion_planning.utils.paths import ONLINE_RESULTS_CANDIDATES_DIR


if test_candidates:
    delta_min = bat_threshold(cand_files)

    for cand_prob in test_candidates:
        # cand_prob = test_candidates[0]
        delta0 = cand_prob.capacity
        bat_out = run_bat_orb(
            cand_prob.source_path,
            capacity_override=delta0,
            delta_min=delta_min,
            output_root=ONLINE_RESULTS_CANDIDATES_DIR
        )
        print("BAT-ORB output path:", bat_out)
    else:
        print("finished the BAT successfully!.")


BAT-ORB output path: /home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/online solutions/candidates/20251125_164202_Town03_Roundabout_10_mid_5_online_BAT-ORB.csv
BAT-ORB output path: /home/abdulrahman/competitive-online-algorithms-motion-planning/results/data/online solutions/candidates/20251125_164202_Town03_Roundabout_15_mid_5_online_BAT-ORB.csv
finished the BAT successfully!.



### ITM-ORB (using helpers, no fallbacks)


In [3]:
from motion_planning.utils.paths import ONLINE_RESULTS_GRAPH_DIR

if test_graphs:
    for gp in test_graphs:
    # gp = test_graphs[0]
        delta0 = gp.capacity
        if delta0 is None:
            delta0 = _parse_risk_budget_from_stem(Path(gp.source_edges_path))
        itm_res = run_itm_online(
            gp.source_edges_path,
            gp.source_timeline_path,
            capacity=delta0,
            psi_t=None,
            gurobi_params=GUROBI_OPTIONS,
            output_root=ONLINE_RESULTS_GRAPH_DIR
        )
        print("ITM-ORB epochs:", len(itm_res))
        if itm_res:
            print("First epoch:", itm_res[0])
else:
    print("No graph problems loaded.")


Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2649771
Academic license 2649771 - for non-commercial use only - registered to ab___@gmail.com
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2649771
Academic license 2649771 - for non-commercial use only - registered to ab___@gmail.com
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2649771
Academic license 2649771 - for non-commercial use only - registered to ab___@gmail.com
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2649771
Academic license 2649771 - for non-commercial use only - registered to ab___@gmail.com
Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2649771
Academic license 2649771 - for non-commercial use only - registered to ab___@gmail.com
ITM-ORB epochs: 5
First epoch: {'epoch': 0, 'start': 'v_0_0', 'goal': 'v_3_5', 'psi': 11.676550869519742, 'sta